# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4f94d19ac0>
├── 'a' --> tensor([[ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698]])
└── 'x' --> <FastTreeValue 0x7f4f94d19940>
    └── 'c' --> tensor([[-0.2000,  0.4455,  1.1107,  1.1576],
                        [ 0.7962,  0.8361,  0.7870,  1.1626],
                        [ 1.2555,  0.3499, -0.3618, -0.1518]])

In [4]:
t.a

tensor([[ 0.2879, -0.6126, -1.2753],
        [-0.9291, -0.4219, -0.3698]])

In [5]:
%timeit t.a

70.4 ns ± 0.0351 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4f94d19ac0>
├── 'a' --> tensor([[-0.0225, -1.0428, -0.6993],
│                   [-0.9829, -1.4477, -1.7987]])
└── 'x' --> <FastTreeValue 0x7f4f94d19940>
    └── 'c' --> tensor([[-0.2000,  0.4455,  1.1107,  1.1576],
                        [ 0.7962,  0.8361,  0.7870,  1.1626],
                        [ 1.2555,  0.3499, -0.3618, -0.1518]])

In [7]:
%timeit t.a = new_value

64.1 ns ± 0.0897 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698]]),
    x: Batch(
           c: tensor([[-0.2000,  0.4455,  1.1107,  1.1576],
                      [ 0.7962,  0.8361,  0.7870,  1.1626],
                      [ 1.2555,  0.3499, -0.3618, -0.1518]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2879, -0.6126, -1.2753],
        [-0.9291, -0.4219, -0.3698]])

In [11]:
%timeit b.a

51.2 ns ± 0.0739 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0034, -0.8646,  0.4241],
               [-1.4877, -0.9079, -2.1445]]),
    x: Batch(
           c: tensor([[-0.2000,  0.4455,  1.1107,  1.1576],
                      [ 0.7962,  0.8361,  0.7870,  1.1626],
                      [ 1.2555,  0.3499, -0.3618, -0.1518]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.345 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 0.335 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 41.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 453 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4f94d15850>
├── 'a' --> tensor([[[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]],
│           
│                   [[ 0.2879, -0.6126, -1.2753],
│                    [-0.9291, -0.4219, -0.3698]]])
└── 'x' --> <FastTreeValue 0x7f4edcfe0f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 60.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4f94d7be80>
├── 'a' --> tensor([[ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698],
│                   [ 0.2879, -0.6126, -1.2753],
│                   [-0.9291, -0.4219, -0.3698]])
└── 'x' --> <FastTreeValue 0x7f4edcff5f40>
    └── 'c' --> tensor([[-0.2000,  0.4455,  1.1107,  1.1576],
                        [ 0.7962,  0.8361,  0.7870,  1.1626],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.1 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]],
       
               [[ 0.2879, -0.6126, -1.2753],
                [-0.9291, -0.4219, -0.3698]]]),
    x: Batch(
           c: tensor([[[-0.2000,  0.4455,  1.1107,  1.1576],
                       [ 0.7962,  0.8361,  0.7870,  1.1626],
                       [ 1.2555,  0.3499, -0.3618, -0.1518]],
         

In [26]:
%timeit Batch.stack(batches)

78 µs ± 70.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698],
               [ 0.2879, -0.6126, -1.2753],
               [-0.9291, -0.4219, -0.3698]]),
    x: Batch(
           c: tensor([[-0.2000,  0.4455,  1.1107,  1.1576],
                      [ 0.7962,  0.8361,  0.7870,  1.1626],
                      [ 1.2555,  0.3499, -0.3618, -0.1518],
                      [-0.2000,  0.4455,  1.1107,  1.1576],
                      [ 0.7962,  

In [28]:
%timeit Batch.cat(batches)

141 µs ± 298 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 2.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
